In [ ]:
import functools
import re
from collections import Counter
from operator import mul
from pathlib import Path

In [ ]:
# MAX_X, MAX_Y = 11, 7
MAX_X, MAX_Y = 101, 103

In [ ]:
type Vector = tuple[int, int]


class Robot:
    """A robot that can move on the tiles."""

    def __init__(self, pos: Vector, speed: Vector):
        """Initialize the robot."""
        self._inital_pos = pos
        self.pos = pos
        self.speed = speed

    def reset(self) -> None:
        """Reset the robot to its initial position."""
        self.pos = self._inital_pos

    def move(self) -> None:
        """Move the robot, wrapping around the board."""
        x, y = self.pos
        dx, dy = self.speed
        self.pos = (x + dx) % MAX_X, (y + dy) % MAX_Y

    def in_quadrant(self) -> int | None:
        """Return the quadrant the robot is in."""
        x, y = self.pos
        left = x < MAX_X // 2
        right = x > MAX_X // 2
        upper = y < MAX_Y // 2
        lower = y > MAX_Y // 2
        if left and upper:
            return 1
        if right and upper:
            return 2
        if left and lower:
            return 3
        if right and lower:
            return 4
        return None

In [ ]:
robots = []
for line in Path("day14_input.txt").read_text().splitlines():
    if numbers := re.findall(r"(-?\d+)", line):
        x, y, vx, vy = map(int, numbers)
        robots.append(Robot((x, y), (vx, vy)))

# Part 1


In [ ]:
for _ in range(100):
    for robot in robots:
        robot.move()

quadrants = Counter(robot.in_quadrant() for robot in robots)
answer = functools.reduce(
    mul, (count for q, count in quadrants.items() if q is not None)
)
answer

# Part 2

Let's not worry too much about the shape or position of the tree. But for a christmas
tree to appear, a lot of robots must have neighbors. For each step, we can count the
number of robots with neighbors and look for a peak in the count.


In [ ]:
def has_neighbors(pos: Vector, positions: set[Vector]) -> bool:
    """Return True if the position has a neighbor."""
    for dx in (-1, 0, 1):
        for dy in (-1, 0, 1):
            if dx == dy == 0:
                continue
            neighbor = (pos[0] + dx, pos[1] + dy)
            if neighbor in positions:
                return True
    return False

In [ ]:
def print_positions(positions: list[Vector]):
    """Print the positions."""
    for y in range(MAX_Y):
        for x in range(MAX_X):
            print("#" if (x, y) in positions else ".", end="")
        print()

In [ ]:
for robot in robots:
    robot.reset()

for i in range(10_000):
    positions = {robot.pos for robot in robots}
    count = sum(has_neighbors(pos, positions) for pos in positions)
    if count > 0.6 * len(robots):
        print(i, count)
        break
    for robot in robots:
        robot.move()